In [1]:
from pathlib import Path

import geopandas as gpd
import pandas as pd
import numpy as np
import xarray as xr
import rioxarray

data_dir = Path('../data')

In [2]:
metrics_dir = data_dir / 'outputs/plots/metrics/x1-y1-z1/net_cdf'

def read_plot_metrics(plot_id: str):
    metrics = xr.open_dataset(metrics_dir / f"{plot_id}.nc", decode_coords='all')
    metrics.load()
    metrics.close()
    return metrics

plots = gpd.read_file(data_dir / "outputs/plots/plots.geojson")
plots = plots.set_index('id')
plots

,site,plot_number,site_plot_id,geometry
id,,,,
AGG_O_01_P1,AGG_O_01,1,AGG_O_01_P1,"POLYGON ((463042.83 5259846.736, 463025.797 52..."
AGG_O_01_P2,AGG_O_01,2,AGG_O_01_P2,"POLYGON ((463124.556 5259819.234, 463116.068 5..."
AGG_O_01_P3,AGG_O_01,3,AGG_O_01_P3,"POLYGON ((463201.174 5259815.806, 463200.551 5..."
AGG_O_01_P4,AGG_O_01,4,AGG_O_01_P4,"POLYGON ((463257.777 5259801.962, 463245.303 5..."
AGG_O_01_P5,AGG_O_01,5,AGG_O_01_P5,"POLYGON ((463303.022 5259789.552, 463289.794 5..."
...,...,...,...,...
ULY_Y_96_P1,ULY_Y_96,1,ULY_Y_96_P1,"POLYGON ((455348.476 5275834.517, 455348.154 5..."
ULY_Y_96_P2,ULY_Y_96,2,ULY_Y_96_P2,"POLYGON ((455296.576 5275837.224, 455299.223 5..."
ULY_Y_96_P3,ULY_Y_96,3,ULY_Y_96_P3,"POLYGON ((455252.092 5275834.16, 455248.849 52..."


In [ ]:
def get_ancillary_metrics(row: gpd.GeoSeries) -> pd.Series:
    id = row.name
    metrics : xr.Dataset = read_plot_metrics(id)    

    return pd.Series({
        "mean_pulse_density": metrics['pulse_density'].mean().item(),
        "total_pulses": metrics['pulse_density'].sum().item(),
        "mean_point_density": metrics['point_density'].mean().item(),
        "total_points": metrics['point_density'].sum().item(),
        "mean_scan_angle": metrics['scan_angle_mean'].mean().item(),
        "max_scan_angle": np.absolute(metrics['scan_angle_mean']).max().item()
    })

In [11]:
get_ancillary_metrics(plots.iloc[0])

mean_pulse_density        48.682927
total_pulses          129740.000000
mean_point_density        94.731332
total_points          252459.000000
mean_scan_angle           -0.070185
max_scan_angle             0.113455
dtype: float64

In [12]:
ancillary_metrics = plots.apply(get_ancillary_metrics, axis=1)
ancillary_metrics

,mean_pulse_density,total_pulses,mean_point_density,total_points,mean_scan_angle,max_scan_angle
id,,,,,,
AGG_O_01_P1,48.682927,129740.0,94.731332,252459.0,-0.070185,0.113455
AGG_O_01_P2,48.289496,114929.0,94.245378,224304.0,-0.065961,0.120404
AGG_O_01_P3,48.290526,137628.0,96.454386,274895.0,-0.066163,0.125832
AGG_O_01_P4,49.795264,130364.0,109.504584,286683.0,-0.064567,0.151091
AGG_O_01_P5,49.103567,114264.0,121.291362,282245.0,-0.060656,0.141343
...,...,...,...,...,...,...
ULY_Y_96_P1,27.347843,69737.0,43.126275,109972.0,-11.770692,25.534655
ULY_Y_96_P2,27.518034,69428.0,39.183908,98861.0,-14.802362,23.922390
ULY_Y_96_P3,26.731061,63513.0,37.543771,89204.0,-15.228471,23.625088


In [ ]:
pd.DataFrame({
    "mean": ancillary_metrics.mean(),
    "min": ancillary_metrics.min(),
    "max": ancillary_metrics.max(),
}, index=['total_points', 'mean_point_density', 'total_pulses', 'mean_pulse_density', 'mean_scan_angle', 'max_scan_angle'])

,mean,min,max
total_points,168056.758730,50080.000000,507081.000000
mean_point_density,67.857449,20.396120,185.416893
total_pulses,80878.295238,38802.000000,230078.000000
mean_pulse_density,32.642827,16.450018,81.530120
mean_scan_angle,1.802210,-25.693419,22.427294
max_scan_angle,13.395605,0.006000,29.005333
